In [1]:
#dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import pymongo
import time
import numpy as np
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
browser = Browser('chrome', headless=True)

In [14]:
#Scrape data and create list from scrapped data

quote_all = []
pages = np.arange(1,11,1)#start page 1, stop page 10, step 1 
quote_id = 0
for page in pages:
    page = requests.get('http://quotes.toscrape.com/page/' +str(page)+ '/')
    soup = bs(page.text,'lxml') 
    quotes = soup.find_all('span', class_='text')
    link_main = soup.find_all('div',class_ = 'quote')
    authors = soup.find_all('small', class_='author')

    i = 1
    for i in range(0,len(quotes)):
        q = quotes[i].text 
        a = authors[i].text
        link = link_main[i].a['href']
        tags = soup.find_all('div', class_='tags')
        t = [ ] 
        for tags in link_main[i].find_all('a' , class_='tag'):
            taggers = tags.text
            t.append(taggers)

        author_url = 'http://quotes.toscrape.com' + link 
        author_info = requests.get('http://quotes.toscrape.com' + link)
        soup = bs(author_info.text,'html.parser')
        author_born = soup.find_all('span', class_='author-born-date')
        author_description = soup.find_all('div', class_='author-description')
        quote_id = quote_id + 1
        data = {
           'quote_id': quote_id,
           'quote': q,
           'author_name': a,
           'tags' : t,
            'dob': author_born[0].text,
            'description': author_description[0].text.replace('\n','')      # removed the new line charcter          
           }
        quote_all.append(data)

    else:
        continue

#Exit browser session which was started for scrapping in hidden mode
browser.quit()

In [6]:
# Setup connection to mongodb
#mongo local connection
#conn = "mongodb://localhost:27017"
#mongo Atlas
conn = "mongo "mongodb+srv://mongocluster.co3vn.mongodb.net/admin" --username admin --password mongo"
client = pymongo.MongoClient(conn)

In [15]:
#Load Quotes data
db = client.quote_etl
db.author_quote.insert_many(quote_all)

Execute postgres_schema.sql in postgres client to create the tables and delete data if exist

In [16]:
#Define postgres connection
conn = f"postgres:Isla@localhost:5432/quotes_db"
engine = create_engine(f'postgresql://{conn}')

In [18]:
engine.table_names()

['author', 'quotes', 'tags']

In [19]:
tags_table = []
quotes_table = []
author_table = []
authors_list = []
for rows in db.author_quote.find({}):
    for tag in rows['tags']:
        tags_table.append({'quote_id':rows['quote_id'] , 'tag' : tag})
    quotes_table.append({'quote_id':rows['quote_id'],'author_name': rows['author_name'],'quote':rows['quote']})       
    if rows['author_name'].strip() not in authors_list:
        authors_list.append(rows['author_name'])
        author_table.append({'author_name':rows['author_name'],'dob':rows['dob'],'description':rows['description']})
        
tags_df = pd.DataFrame(tags_table)
quotes_df = pd.DataFrame(quotes_table)
author_df = pd.DataFrame(author_table)

In [20]:
#Load tags data
tags_df.to_sql(name='tags', con=engine, if_exists='append', index=False)

In [21]:
#Load quotes data
quotes_df.to_sql(name='quotes', con=engine, if_exists='append', index=False)

In [22]:
#Load author data
author_df.to_sql(name='author', con=engine, if_exists='append', index=False)